In [1]:
!pip install sentence-transformers googletrans==3.1.0a0

In [10]:
from grammar_ru.corpus import ParallelCorpus
from utils.make_match import make_match
from ca import Seq2VecMatcher
import os
from pathlib import Path

In [16]:
parallel_corpus = ParallelCorpus(Path('./files/parallel_corpus.zip'))
en_book_relations = parallel_corpus.en_book.get_relations()
en_book_relations = en_book_relations[en_book_relations['relation_name'] == 'en_book_en_retell']

In [12]:
finished_tasks_num = 0
err_relations = []
matcher = Seq2VecMatcher()

In [13]:
en_book_relations.iloc[1:4, 0].to_list()

['95563950-9e63-46a0-99df-e6c5b2e4528a',
 '4be61168-5f93-48c5-b96e-dd4346a8f2c4',
 '0f96c70e-638b-414c-abe7-117595c52b56']

In [14]:
file_path = f"data/result_dataset.json"
os.makedirs(os.path.dirname(file_path), exist_ok=True)

with open(file_path, 'a', encoding='utf-8') as file:
    file.write('[')

In [ ]:
for i in range(len(en_book_relations)):
    print(f"working on {i + 1}/{len(en_book_relations)}", end='\r')

    book_id = en_book_relations.iloc[i, 0]
    retell_id = en_book_relations.iloc[i, 1]

    book_chapter_df = parallel_corpus.en_book.get_frames(uids=[book_id]).first()
    retell_chapter_df = parallel_corpus.en_retell.get_frames(uids=[retell_id]).first()
    
    try:
        sent_dataset_json = make_match(book_chapter_df, retell_chapter_df, matcher)

        with open(file_path, 'a', encoding='utf-8') as file:
            file.write(sent_dataset_json[1:-1])
            if i != len(en_book_relations) - 1:
                file.write(',')
        
        print("                                                                    ", end='\r')
    except Exception as ex:
        err_relations.append(((book_id, retell_id), ex))

with open(file_path, 'a', encoding='utf-8') as file:
    file.write(']')

In [31]:
raise err_relations[0][1]

IndexError: index 10 is out of bounds for axis 0 with size 10

In [ ]:
import json

data = None

with open(file_path, 'r') as file:
    data = json.load(file)

print()